In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import random
import os
import pickle
import matplotlib.pyplot as plt
import time
import shutil
import getpass
import multiprocessing
import copy

from subprocess import Popen, PIPE
from threading import Timer
from sklearn.datasets import make_spd_matrix
from scipy.stats import random_correlation
from statsmodels.discrete.discrete_model import NegativeBinomialP
import statsmodels.api as sm
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

date = str(datetime.date.today())

In [ ]:
TEST = False
CPU = min(15, multiprocessing.cpu_count() - 1)

In [ ]:
global models
models = {}

In [ ]:
pth = '../data/gencode.v22.annotation.gene.probeMap'
probemap = pd.read_csv(pth, sep='\t')
genemap = probemap[["id", 'gene']].set_index("id").to_dict()["gene"]
probemap["length"] = probemap["chromEnd"] - probemap["chromStart"]
genelengths = probemap.groupby("gene").mean()["length"]
genekb = genelengths / 1000

In [ ]:
pth = '../data/TARGET-WT.htseq_counts.tsv.gz'
lc1 = pd.read_csv(pth, sep='\t', index_col=0)
lc1 = (2**lc1) - 1

lc1 = lc1.reset_index()
lc1["hugo"] = lc1["Ensembl_ID"].map(genemap).values
lc1 = lc1.dropna()
lc1 = lc1.drop("Ensembl_ID", axis=1)
exp = lc1.groupby("hugo").sum()
thresh80 = np.percentile(exp.mean(axis=1).sort_values(ascending=False).values, 80)
exp = exp[exp.mean(axis=1) > thresh80]

In [ ]:
def get_nb(y):
    x = np.ones_like(y)
    res = sm.NegativeBinomial(y, x, loglike_method='nb2').fit(disp=0)
    
    mu = np.exp(res.params[0])
    alpha = res.params[1]
    size = 1. / alpha
    prob = size / (size + mu)
    return stats.nbinom(size, prob)

In [ ]:
import errno    
import os

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

In [ ]:
pth = '../data/h.all.v6.2.symbols.gmt'
mapper = {}
gs_genes = set()
with open(pth) as f:
    for line in f:
        elements = line.strip().split('\t')
        name = elements[0]
        if 'HALLMARK' in name:
            desc = elements[1]
            genes = elements[2:]
            mapper[name] = [g for g in genes if g in exp.index.values]
            gs_genes.update(mapper[name])

In [ ]:
def cohend(d1, d2):
    """
    https://machinelearningmastery.com/effect-size-measures-in-python/
    """
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

def run(cmd, timeout_sec=900):
    """
    https://stackoverflow.com/questions/1191374/using-module-subprocess-with-timeout
    :param cmd:
    :param timeout_sec:
    :return:
    """
    proc = Popen(cmd, stdout=PIPE, stderr=PIPE)
    timer = Timer(timeout_sec, proc.kill)

    try:
        timer.start()
        stdout, stderr = proc.communicate()

    finally:
        timer.cancel()

    return stdout, stderr

def run_hydra(data_pth, gs, database, output):
    cmd = ['docker',
           'run',
           '-v', '%s:/data' % os.getcwd(),
           'jpfeil/hydra@sha256:de39cd5b3b04b4ff87e8751084a21e743f25af431048bddbfe2faded05268467',
           'sweep',
           '-e', data_pth,
           '--gmt', database,
           '--gmt-regex', gs,
           '--min-prob-filter', '0.05',
           '--min-mean-filter', '1.0',
           '--min-gene-filter', '1',
           '--sensitive',
           '--gamma', '5.00',
           '--sF', '2.0',
           '-K', '2',
           '--CPU', str(CPU),
           '--debug',
           '--output-dir', output]
    
    stdout, stderr = run(cmd)
    
    assert os.path.exists(output)
    
    return stdout, stderr
    
def run_gsea(data_pth, output_pth, database):
    cmd = ['docker',
           'run',
           '-v', '%s:/data' % os.getcwd(),
           'jpfeil/pyrwrapper@sha256:a765a8f284c94b7791cfe87b8b83b34594a609873bca755b2fb83e45b5dd56fc',
           'GSEA-manager.py',
           data_pth, 
           output_pth,
           database]
    
    stdout, stderr = run(cmd)
    assert os.path.exists(output_pth)
    return stdout, stderr


def AlmightyCorrcoefEinsumOptimized(O, P):
    """
    https://github.com/ikizhvatov/efficient-columnwise-correlation/blob/master/columnwise_corrcoef_perf.py
    """
    (n, t) = O.shape      # n traces of t samples
    (n_bis, m) = P.shape  # n predictions for each of m candidates

    DO = O - (np.einsum("nt->t", O, optimize='optimal') / np.double(n)) # compute O - mean(O)
    DP = P - (np.einsum("nm->m", P, optimize='optimal') / np.double(n)) # compute P - mean(P)

    cov = np.einsum("nm,nt->mt", DP, DO, optimize='optimal')

    varP = np.einsum("nm,nm->m", DP, DP, optimize='optimal')
    varO = np.einsum("nt,nt->t", DO, DO, optimize='optimal')
    tmp = np.einsum("m,t->mt", varP, varO, optimize='optimal')
    return cov / np.sqrt(tmp)
    
    
def corr2cov(corr, std):
    return corr * np.outer(std, std)

def get_log2tpm1(og, mod):
    denom = genekb.reindex(og.index)
    rpk = og.divide( denom, axis=0 )
    scale = rpk.sum(axis=0) / 1000000.
    
    mod_rpk = mod.divide( denom, axis=0 )
    tpm = mod_rpk.divide(scale, axis=1)    
    return np.log2( tpm + 1 )

def get_model_callback(results):
    models[results[0]] = results[1]

def nb_runner(gene, exp):
    return gene, get_nb(exp.loc[gene, :].values)
    
def get_data(models, targets, size, 
             eff, pdiff, ptype):
    
    print("Creating test data...")
    ssize = int(ptype * size)
    bsize = size - ssize
    
    bsamples = ['normal%d' % x for x in range(bsize)]
    ssamples = ['active%d' % x for x in range(bsize, size)]

    samples = bsamples + ssamples
    assert len(samples) == size
    
    genes = list(models.keys())
        
    train = pd.DataFrame(index=genes,
                         columns=samples)
        
    test = pd.DataFrame(index=genes,
                        columns=samples)
    
    print("Sampling from TRAIN and TEST distributions")
    for gene in genes:   
        train.loc[gene, :] = models[gene].rvs(size)
        test.loc[gene, :] = models[gene].rvs(size)
        
    og_train = copy.deepcopy(train)
    og_test = copy.deepcopy(test)
    
    print("Randomly sampling DEGs")
    degs = np.random.choice(targets, 
                            int(pdiff * len(targets)), 
                            replace=False)
    
    print("Creating subtype mean expression vector")
    for i, gene in enumerate(degs):
        mu = models[gene].mean()
        s = models[gene].std()
        
        mu2 = (eff * s) + mu
        
        size = models[gene].args[0]
        prob = models[gene].args[1]
    
        train.loc[gene, ssamples] = stats.nbinom(size, prob, loc=mu2).rvs(ssize)
        test.loc[gene, ssamples] = stats.nbinom(size, prob, loc=mu2).rvs(ssize)
      
    train_log2TPM = get_log2tpm1(og_train, train)
    test_log2TPM = get_log2tpm1(og_test, test)
    
    cds = []
    for gene in degs:
        cd = cohend(train_log2TPM.loc[gene, ssamples].values,
                    train_log2TPM.loc[gene, bsamples].values)
        cds.append(cd)
        
    return train_log2TPM, test_log2TPM, degs, np.mean(cds)

In [ ]:
def validate(in_dir, out_dir, gs, gs_genes, data, 
             effect, percent_diff, models, size=300):
    
    # One criticism was that I picked too many magic 
    # numbers, so here I'm going to pick a random number
    subtype_frac = round(random.uniform(0.15, 0.4), 2)
    
    tag = 'eff-%.2f-diff-%.2f-frac-%.2f-size-%d' % (effect, 
                                                    percent_diff, 
                                                    subtype_frac, 
                                                    size)
    print(tag)
    
    train_pth = os.path.join(in_dir, 
                             'synthetic-%s-train-%s-%s.tsv' % (gs, 
                                                               tag, 
                                                               date))
    
    test_pth = os.path.join(in_dir, 
                            'synthetic-%s-test-%s-%s.tsv' % (gs, 
                                                             tag, 
                                                             date))
    
    
    deg_pth = os.path.join(in_dir,
                           'synthetic-%s-degs-%s-%s.tsv' % (gs, 
                                                            tag,
                                                            date))
    
    cohen_pth = os.path.join(in_dir, 
                             'synthetic-%s-cohen-%s-%s.tsv' % (gs, 
                                                               tag,
                                                               date))
    
    
    train, test, degs, cohen = get_data(models, 
                                        gs_genes,
                                        size, 
                                        effect, 
                                        percent_diff, 
                                        subtype_frac)
    
        
    train.to_csv(train_pth, sep='\t')
    test.to_csv(test_pth, sep='\t')
    
    with open(deg_pth, 'w') as f:
        f.write('\n'.join(degs))
        
    with open(cohen_pth, 'w') as f:
        f.write(str(cohen))
    
    hydra_pth = os.path.join(out_dir, 'Hydra', tag, gs)
    
    tic = time.perf_counter()
    stdout, stderr = run_hydra(train_pth, gs, '../data/h.all.v6.2.symbols.gmt', hydra_pth)
    toc = time.perf_counter()
    
    time_dir = os.path.join(out_dir, 'TIME', 'Hydra', tag)
    mkdir_p(time_dir)
    time_pth = os.path.join(time_dir, gs)
    with open(time_pth, 'w') as f:
        f.write(str(toc - tic))
    
    gsea_pth = os.path.join(out_dir)
    run_gsea(test_pth, gsea_pth, '../data/h.all.v6.2.symbols.gmt')
    

def run_validation(in_dir,
                   out_dir,
                   data, 
                   gene_sets, 
                   effects, 
                   percent_diffs,
                   models,
                   sizes):   
    
    for size in sizes:
        for effect in effects:
            for percent_diff in percent_diffs:
                # Sample gene sets to save time
                if len(gene_sets) > 10:
                    gs_subsets = random.sample(gene_sets.keys(), k=10)
            
                else:
                    gs_subsets = gene_sets.keys()
                
                for gs in gs_subsets:
                    validate(in_dir, 
                             out_dir,
                             gs, 
                             gene_sets[gs],
                             data, 
                             effect, 
                             percent_diff,
                             models,
                             size)

In [ ]:
if TEST:
    in_dir = 'test_input/%s' % date
    out_dir = 'test_output/%s' % date
    
    if os.path.exists(in_dir):
        shutil.rmtree(in_dir)
    if os.path.exists(out_dir):
        shutil.rmtree(out_dir)
    
    mkdir_p(in_dir)
    mkdir_p(out_dir)
    
    _exp = exp.reindex(mapper['HALLMARK_ESTROGEN_RESPONSE_EARLY']).dropna()
    test = {'HALLMARK_ESTROGEN_RESPONSE_EARLY': [x for x in mapper['HALLMARK_ESTROGEN_RESPONSE_EARLY']]}
    
    models = {}
    for gene in _exp.index.values:
        models[gene] = get_nb(_exp.loc[gene, :].values)
    
    print("Staring Run...")
    run_validation(in_dir,
                   out_dir,
                   _exp, 
                   test,
                   [1.5],
                   [0.10, 0.25], 
                   models,
                   [300])

In [ ]:
# The real deal
if not TEST:
    in_dir = '../data/input/%s' % date
    out_dir = '../data/output/%s' % date
    mkdir_p(in_dir)
    mkdir_p(out_dir)

    with open(os.path.join(out_dir, 'V17'), 'a') as f:
        f.write(date)

    models = {}
    for gene in exp.index.values:
        models[gene] = get_nb(exp.loc[gene, :].values)
          
    assert len(models) == len(exp)
    run_validation(in_dir, 
                   out_dir,
                   exp, 
                   mapper,
                   [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 2.5, 3.0],
                   [0.25, 0.1],
                   models,
                   [50, 100, 200, 400, 800])